<img src="Imagenes/Mac_wallpaper_3.png" width="50%">

In [ ]:
!pip install "qiskit[visualization]" --user

In [ ]:
!pip install qiskit-aer --user

# Retroceso de fase

Como vimos en el notebook anterior, podemos utilizar una compuerta CNOT para crear entrelazamiento entre dos qubits cuando uno de ellos se encuentra en superposición. Ahora nos hacemos la pregunta ¿Qué pasaría si ambos qubits están en superposición? ¿La compuerta CNOT tendrá algún efecto interesante? Bueno, empecemos evaluando el siguiente estado

$\hspace{9 cm} |++\rangle = \dfrac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle)$

Si ahora le aplicamos una compuerta CNOT, es fácil ver que sin importar qué qubit escojamos como el control, el estado no cambia en absoluto. Esto lo podemos comprobar con el simulador.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_bloch_multivector, array_to_latex

qc = QuantumCircuit(2)

qc.h(0)
qc.h(1)

job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_statevector(qc,precision).data

display(array_to_latex(vector_estado, prefix="|++\\rangle ="))

In [ ]:
qc = QuantumCircuit(2)

qc.h(0)
qc.h(1)

qc.cx(0,1)

job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_statevector(qc,precision).data

display(array_to_latex(vector_estado))

Ahora, hagamos que uno de los qubits se encuentre en el estado $|-\rangle$:

$\hspace{9 cm} |+-\rangle = \dfrac{1}{2}(|00\rangle - |01\rangle + |10\rangle - |11\rangle)$

Si ahora aplicamos una compuerta CNOT tomando al qubit en el estado $|-\rangle$ como el objetivo (es decir, el qubit sobre el que se aplica la compuerta NOT) vemos que nuestro estado se convierte en

$\hspace{8 cm} CNOT|+-\rangle = \dfrac{1}{2}(|00\rangle - |01\rangle + |11\rangle - |10\rangle)$

Notamos que los estados $|11\rangle$ y $|10\rangle$ intercambiaron sus fases. Ahora, intentemos reescribir este nuevo estado completo como el producto tensorial de los estados individuales de nuestros qubits para ver qué efecto tuvo este cambio. 

$\hspace{3 cm} \dfrac{1}{2}(|00\rangle - |01\rangle + |11\rangle - |10\rangle) = \dfrac{1}{2}[|0\rangle(|0\rangle - |1\rangle) - |1\rangle(|0\rangle - |1\rangle)] = \dfrac{1}{2}(|0\rangle - |1\rangle)\otimes(|0\rangle - |1\rangle) = |--\rangle$

¡Alteramos el estado del qubit control! A pesar de que fue sobre el qubit en el estado $|-\rangle$ que aplicamos la compuerta NOT, el resultado fue alterar el estado del qubit $|+\rangle$, cambiándolo al estado $|-\rangle$, mientras que el estado de nuestro qubit objetivo se mantuvo intacto.

In [ ]:
qc = QuantumCircuit(2)

qc.x(0)
qc.h(0)
qc.h(1)

job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_statevector(qc,precision).data

display(array_to_latex(vector_estado, prefix="|+-\\rangle ="))
plot_bloch_multivector(vector_estado,reverse_bits=True)

In [ ]:
qc = QuantumCircuit(2)

qc.x(0)
qc.h(0)
qc.h(1)

qc.cx(1,0)

job = execute(qc,Aer.get_backend('statevector_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_statevector(qc,precision).data

display(array_to_latex(vector_estado))
plot_bloch_multivector(vector_estado,reverse_bits=True)

Este fenómeno se conoce como **Retroceso de fase** (*Phase Kickback* en inglés), pues de cierta forma se ha convertido el efecto de la compuerta NOT sobre el qubit objetivo en un cambio de fase local que luego se "retrocede" al qubit control (recordemos que la diferencia entre los estados $|+\rangle$ y $|-\rangle$ es su fase local).

La importancia del retroceso de fase radica en que es un recurso muy utilizado en diversos algoritmos cuánticos.

### Invertir una compuerta CNOT

Continuando con lo que hemos visto, y como un ejemplo de la utilidad del retroceso de fase, vamos a ver cómo podemos usar dicho recurso para invertir una compuerta CNOT. 

Con invertir, nos referimos a intercambiar el qubit control con el qubit objetivo. Es decir, convertir una compuerta `.cx(i,j)` en una compuerta `.cx(j,i)`.

El ejemplo que vimos anteriormente nos puede ayudar a vislumbrar una forma de lograr esto. Como pudimos ver, si aplicamos una compuerta CNOT al estado $|+-\rangle$ usando al qubit $q_{0}$ como el objetivo obtenemos el estado $|--\rangle$. Ahora, apliquemos una compuerta Hadamard a ambos estados, recordando que dicha compuerta es su propio inverso

$\hspace{10 cm}H_{1}H_{0}|--\rangle = |11\rangle$

Y si recordamos cómo obtener el estado $|+-\rangle$ utilizando compuertas Hadamard, podemos ver el estado inicial de nuestros qubits

$\hspace{10 cm}H_{1}H_{0}|01\rangle = |+-\rangle$

Juntando todas estas operaciones, notamos que nuestro input es el estado $|01\rangle$ y nuestro output es el estado $|11\rangle$ ¡Nuestro circuito completo aplica una compuerta CNOT que utiliza el qubit $q_{1}$ como el objetivo!

Y esto se puede probar para los otros cuatro estados restantes, de maner que colocar compuertas Hadamard antes y después de una compuerta CNOT invierte la dirección en la que esta se aplica. Usemos el simulador para comprobarlo, imprimiendo el operador completo que actúa sobre el circuito en ambos casos.

In [ ]:
qc = QuantumCircuit(2)

qc.h(0)
qc.h(1)

#Compuerta CNOT que utiliza el qubit q_1 como el objetivo
qc.cx(0,1)

qc.h(0)
qc.h(1)

qc.draw(output="mpl")

In [ ]:
job = execute(qc,Aer.get_backend('unitary_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_unitary(qc,precision).data

display(array_to_latex(vector_estado, prefix="Operador ="))

In [ ]:
qc = QuantumCircuit(2)

#Compuerta CNOT que utiliza el qubit q_0 como el objetivo
qc.cx(1,0)

qc.draw(output="mpl")

In [ ]:
job = execute(qc,Aer.get_backend('unitary_simulator'),optimization_level=0)
precision = 3 
vector_estado = job.result().get_unitary(qc,precision).data

display(array_to_latex(vector_estado, prefix="Operador ="))

Esta igualdad resulta muy útil cuando queremos ejecutar un circuito en una unidad de hardware que solo permite aplicar compuertas CNOT en una dirección, pues nos permite aplicar compuertas CNOT en ambas direcciones.